In [1]:
# import
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
import time
from tqdm import tqdm_notebook

from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException
import requests
import urllib.parse

# kyuri 브랜치에 있는 justwatch_2.xlsx 이 파일로 시도해보기!!!

In [2]:

# 엑셀 파일을 읽어옵니다.
just = pd.read_excel('../data/justwatch_2.xlsx', index_col=0)
just.head()

,title,original_title,year,season_episode,runtime,genre,age_rating,Production country
0,Ojingeo Geim,Squid Game,2021,2,55min,"액션, 드라마, 스릴러",19,대한민국
1,12인의 심판자,The Twelve,2019,2,54min,"드라마, 스릴러",NaN,벨기에
2,희생자 게임,誰是被害者,2020,2,1시간 0분,"드라마, 범죄, 스릴러",19,대만
3,스위트 투스: 사슴뿔을 가진 소년,Sweet Tooth,2021,3,52min,"SF, 드라마, 판타지, 액션",15,미국
4,Sweet Home,NaN,2020,3,1시간 0분,"스릴러, 드라마, 판타지, 공포, SF, 범죄",NaN,대한민국


In [16]:
def get_imdb_result(query):
    # NaN 값인 경우 처리합니다.
    if pd.isna(query):
        return None
    
    # 검색 쿼리를 URL 인코딩합니다.
    query = urllib.parse.quote(query + " TV IMDb")
    
    # 구글 검색 URL을 구성합니다.
    url = f"https://www.google.com/search?q={query}"
    
    # 요청 헤더를 설정합니다.
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
    }
    
    # 구글 검색 페이지에 요청을 보냅니다.
    response = requests.get(url, headers=headers)
    
    # 응답의 상태 코드를 확인합니다.
    if response.status_code == 200:
        # HTML 내용을 파싱합니다.
        soup = BeautifulSoup(response.text, "html.parser")
        
        # 모든 검색 결과를 가져옵니다.
        search_results = soup.find_all("div", class_="g")
        
        # IMDb 사이트의 URL을 포함한 첫 번째 검색 결과를 찾습니다.
        for result in search_results:
            link = result.find("a")
            if link and 'href' in link.attrs:
                url = link['href']
                # IMDb 사이트의 URL인지 확인합니다.
                if "imdb.com" in url:
                    title = result.find("h3").text
                    return {"title": title, "url": url}
    
    return None

In [6]:
# 기존 df에 넣기
def get_imdb_results(df):
    imdb_titles = []
    imdb_urls = []
    for i, row in df.iterrows():
        query = str(row["original_title"]) + str(row["title"]) + str(row["year"])  # NaN 값이 있을 수 있으므로 문자열로 변환합니다.
        result = get_imdb_result(query)
        if result:
            imdb_titles.append(result["title"])
            imdb_urls.append(result["url"])
        else:
            imdb_titles.append(None)
            imdb_urls.append(None)
    return imdb_titles, imdb_urls

# IMDb 제목과 URL을 가져와서 just 데이터프레임에 새로운 열로 추가합니다.
just["IMDb_title"], just["IMDb_URL"] = get_imdb_results(just)

ValueError: Length of values (600) does not match length of index (1090)

In [5]:
# 저장
just.to_csv('../data/justwatch_url.csv', sep="/", encoding="utf-8")

just.to_excel('../data/justwatch_url.xlsx')

---

In [9]:
# 데이터프레임에 있는 각 행의 내용에 대해 IMDb 검색 결과를 가져옵니다.
def get_imdb_results_df(df):
    imdb_results = []
    for i, row in df.iterrows():
        query = str(row["original_title"]) + str(row["title"]) + str(row["year"])  # NaN 값이 있을 수 있으므로 문자열로 변환합니다.
        result = get_imdb_result(query)
        imdb_results.append(result)
    return imdb_results

# 데이터프레임의 처음 5개 행에 대한 IMDb 검색 결과를 가져옵니다.
imdb_results_df = get_imdb_results_df(just)

In [20]:
imdb_results_df[:5]

[None, None, None, None, None]